In [14]:
## python 3.7
import re
import time
#!pip install pymysql
import pymysql
#!pip install pandas
#!pip install openpyxl
import pandas as pd
#!pip install translate
import translate
from translate import *
from numpy import *
#!pip install unidecode
import unidecode
#!pip install snowballstemmer
import snowballstemmer
import numpy as np
#!pip install Dicio
from dicio import Dicio

#!pip install nltk
import nltk
## download somente a primeira vez:
#nltk.download('rslp')

In [17]:
def MapClinCIPE(TermsFile,CIPEFile,MRCONSOFILE,MRXW_PORFILE,MRRELFILE,OutputFile):
    #https://github.com/dmort27/panphon/blob/master/panphon/distance.py
    def levenshtein(source, target):
        if len(source) < len(target):
            return levenshtein(target, source)

        # So now we have len(source) >= len(target).
        if len(target) == 0:
            return len(source)

        # We call tuple() to force strings to be used as sequences
        # ('c', 'a', 't', 's') - numpy uses them as values by default.
        source = np.array(tuple(source))
        target = np.array(tuple(target))

        # We use a dynamic programming algorithm, but with the
        # added optimization that we only need the last two rows
        # of the matrix.
        previous_row = np.arange(target.size + 1)
        for s in source:
            # Insertion (target grows longer than source):
            current_row = previous_row + 1

            # Substitution or matching:
            # Target and source items are aligned, and either
            # are different (cost of 1), or are the same (cost of 0).
            current_row[1:] = np.minimum(current_row[1:], np.add(previous_row[:-1], target != s))

            # Deletion (target grows shorter than source):
            current_row[1:] = np.minimum(current_row[1:],current_row[0:-1] + 1)

            previous_row = current_row

        return previous_row[-1]

    def comb(lis, dict_lis):
        aux = []
        for s in lis:
            for ss in dict_lis:
                aux.append(s+' '+ss)
        return aux

    def removearticles1(text):
        text = re.sub('"','',text)
        return re.sub('\s+(de|do|dos|da|das|o|os|a|as|com|para|pra|e|ou|na|nas|no|nos|ao|aos|em|mg|ml|um)(\s+)',' ', text)

    def removearticles2(text):
        text = re.sub('"','',text)
        #text = re.sub('^(\s*)(de|do|dos|da|das|o|os|a|as|com|para|pra|e|ou|na|nas|no|nos|ao|aos|em|mg|ml)(\s+)','',text)
        return re.sub('((^\s*)|(\s+))(de|do|dos|da|das|o|os|a|as|com|para|pra|e|ou|na|nas|no|nos|ao|aos|em|mg|ml)(($\s*)|(\s+))', ' ', text)

    def tem_num(s):
        return bool(re.match('.*\d.*', s))

    def removearticles3(text):
        text = re.sub('"','',text)
        #text = re.sub('^(\s*)(de|do|dos|da|das|o|os|a|as|com|para|pra|e|ou|na|nas|no|nos|ao|aos|em|mg|ml)(\s+)','',text)
        return re.sub('((^\s*)|(\s+))(de|do|dos|da|das|o|os|a|as|com|para|pra|e|ou|na|nas|no|nos|ao|aos|em|mg|ml)(($\s*)|(\s+))', ' ', text)

    #hunspell tem problemas com windows, muito ruim de instalar
    #funcao antiga
    #def lematizador(text):
    #    auxt = str('')
    #    words = text.split()
    #    for x in words:
    #        if h.spell(x):
    #            if len(h.stem(x)) != 0:
    #                auxt += h.stem(x)[0] + ' '
    #            else:
    #                auxt += x + ' '
    #        else:
    #            auxt += x + ' '
    #    auxt.strip()
    #    return auxt

    #lematizer e sent_tokenizer precisam ficar fora para salvar tempo, only load 1 time
    lemmatizer = nltk.WordNetLemmatizer()
    sent_tokenizer=nltk.data.load('tokenizers/punkt/portuguese.pickle')
    def lematizador(raw_text):
        text = sent_tokenizer.tokenize(raw_text)
        #stemmer = nltk.stem.RSLPStemmer()
        auxt = ""
        words = text
        for x in words:
            #stemizado = stemmer.stem(x)
            lemma = lemmatizer.lemmatize(x)
            if lemma != 0:
                auxt = lemma + " "
            else:
                auxt = x + " "
        auxt.strip()
        return auxt

    def stemmer(text):
        stemmmmer = snowballstemmer.stemmer('portuguese')
        return ' '.join(stemmmmer.stemWords(text.split())).strip()


    #h = Hunspell('pt_BR', hunspell_data_dir='pt_BR')

    #read = pd.ExcelFile('Banco de Termos final Lucas 10-04-2019.xlsx') #antiga, erro, nao existe Planilha1 no arquivo
    read = pd.ExcelFile(TermsFile)
    df_termos = read.parse('Planilha1')

    #read = pd.ExcelFile(CIPE_df_file)
    #CIPE_df = read.parse('Match')

    read = pd.ExcelFile(CIPEFile)
    df_CIPE = read.parse('Planilha1')



    f = open(MRXW_PORFILE, "r")
    MRXW_PORRAW = f.read().split("\n")
    del MRXW_PORRAW[-1] #ultima linha é vazia
    f.close()
    #print(len(MRXW_PORRAW)) #1589110
    #print(MRXW_PORRAW[123400]) #'POR|amostra|C0550117|L12632755|S15615562|'

    CUI2 = []
    WD2 = []
    for y,linha in enumerate(MRXW_PORRAW):
        linhaSeparada =  linha.split("|")
        CUI = linhaSeparada[2]
        WD  = linhaSeparada[1]
        CUI2.insert(y,CUI) #cui
        WD2.insert(y,WD)   #word in lower case

    umls_word = np.array([WD2,CUI2])
    umls2_word = np.array([WD2,WD2,WD2,CUI2])
    df_umls_word = pd.DataFrame(umls_word.transpose(),columns=['WD','CUI'])
    df_umls_word_norm = pd.DataFrame(umls2_word.transpose(),columns=['WD', 'WDlemma', 'WDstem', 'CUI'])
    del CUI2
    del WD2


    with open(MRCONSOFILE, "r", encoding="utf8") as MRCONSORAW:
        # C0003803|SPA|S|L3401341|PF|S3928999|Y|A9197782||M0001704|D001139|
        # MSHSPA|MH|D001139|Malformación de Arnold-Chiari|3|N||

        CUI1 = []
        STR1 = []
        for y,linha in enumerate(MRCONSORAW):
            linhaSeparada =  linha.split("|")
            CUI = linhaSeparada[0]
            STR  = linhaSeparada[14]
            LAT  = linhaSeparada[1]
            if LAT == "POR":
                CUI1.insert(y,CUI)
                STR1.insert(y,STR)

    umls = np.array([STR1,CUI1])
    umls2 = np.array([STR1,STR1,STR1,CUI1])
    df_umls = pd.DataFrame(umls.transpose(),columns=['STR','CUI'])
    df_umls_norm = pd.DataFrame(umls2.transpose(),columns=['STR', 'STRlemma', 'STRstem', 'CUI'])
    del CUI1
    del STR1
    

    with open(MRRELFILE, "r", encoding="utf8") as MRRELRAW:
        # C0851745|A31370316|SDUI|CHD|C0003466|A31264097|SDUI||R191307594||MDRKOR||||N||

        CUIs2 = []
        CUIs1 = []
        for y,linha in enumerate(MRRELRAW):
            linhaSeparada =  linha.split("|")
            CUI1 = linhaSeparada[0]
            CUI2  = linhaSeparada[4]
            REL = linhaSeparada[3]
            if REL == "SY" and CUI1 != CUI2:
                CUIs1.insert(y,CUI1)
                CUIs2.insert(y,CUI2)

    umls_word = np.array([CUIs2,CUIs1])
    df_umls_rel = pd.DataFrame(umls_word.transpose(),columns=['CUI1','CUI2'])
    del CUIs1
    del CUIs2


    for x in range(0,df_umls['STR'].size):
        df_umls.at[x,'STR'] = str(df_umls.at[x,'STR']).lower()

    for x in range(0,df_umls_word['WD'].size):
        df_umls_word.at[x,'WD'] = str(df_umls_word.at[x,'WD']).lower()

    for x in range(0,df_umls_norm['STR'].size):
        df_umls_norm.at[x,'STR'] = str(df_umls_norm.at[x,'STR']).lower()
        df_umls_norm.at[x,'STRlemma'] = str(df_umls_norm.at[x,'STRlemma']).lower()
        df_umls_norm.at[x,'STRstem'] = str(df_umls_norm.at[x,'STRstem']).lower()

    for x in range(0,df_umls_word_norm['WD'].size):
        df_umls_word_norm.at[x,'WD'] = str(df_umls_word_norm.at[x,'WD']).lower()
        df_umls_word_norm.at[x,'WDlemma'] = str(df_umls_word_norm.at[x,'WDlemma']).lower()
        df_umls_word_norm.at[x,'WDstem'] = str(df_umls_word_norm.at[x,'WDstem']).lower()


    for x in range(0,df_umls_norm['STR'].size):
        df_umls_norm.at[x,'STR'] = removearticles1(unidecode.unidecode(str(df_umls_norm.at[x,'STR']))).strip()

    for x in range(0,df_umls_norm['STRlemma'].size):
        df_umls_norm.at[x,'STRlemma'] = removearticles1(unidecode.unidecode(lematizador(str(df_umls_norm.at[x,'STRlemma']))))

    for x in range(0,df_umls_norm['STRstem'].size):
        df_umls_norm.at[x,'STRstem'] = removearticles1(unidecode.unidecode(stemmer(str(df_umls_norm.at[x,'STRstem']))))

    for x in range(0,df_umls_word_norm['WD'].size):
        df_umls_word_norm.at[x,'WD'] = unidecode.unidecode(str(df_umls_word_norm.at[x,'WD'])).strip()

    for x in range(0,df_umls_word_norm['WDlemma'].size):
        df_umls_word_norm.at[x,'WDlemma'] = unidecode.unidecode(lematizador(str(df_umls_word_norm.at[x,'WDlemma'])))

    for x in range(0,df_umls_word_norm['WDstem'].size):
        df_umls_word_norm.at[x,'WDstem'] = unidecode.unidecode(stemmer(str(df_umls_word_norm.at[x,'WDstem'])))

    for x in range(0,df_termos.size):
        df_termos.at[x,'Termos'] = str(df_termos.at[x,'Termos']).lower()

    for x in range(0,df_CIPE['Termo'].size):
        df_CIPE.at[x,'Termo'] = str(df_CIPE.at[x,'Termo']).lower()    

    df_termos_norm = pd.DataFrame(data=None, columns=['Termos'])
    df_termos_lemma = pd.DataFrame(data=None, columns=['Termos'])
    df_termos_stem = pd.DataFrame(data=None, columns=['Termos'])

    df_CIPE_norm = pd.DataFrame(data=None, columns=['Termo'])
    df_CIPE_lemma = pd.DataFrame(data=None, columns=['Termo'])
    df_CIPE_stem = pd.DataFrame(data=None, columns=['Termo'])

    for x in range(0,df_termos.size):
        df_termos_norm.at[x,'Termos'] = removearticles1(unidecode.unidecode(str(df_termos.at[x,'Termos']))).strip()

    for x in range(0,df_termos.size):
        df_termos_lemma.at[x,'Termos'] = removearticles1(unidecode.unidecode(lematizador(str(df_termos.at[x,'Termos']))))

    for x in range(0,df_termos.size):
        df_termos_stem.at[x,'Termos'] = removearticles1(unidecode.unidecode(stemmer(str(df_termos.at[x,'Termos']))))    

    for x in range(0,df_CIPE['Termo'].size):
        df_CIPE_norm.at[x,'Termo'] = removearticles1(unidecode.unidecode(str(df_CIPE.at[x,'Termo']))).strip()

    for x in range(0,df_CIPE['Termo'].size):
        df_CIPE_lemma.at[x,'Termo'] = removearticles1(unidecode.unidecode(lematizador(str(df_CIPE.at[x,'Termo']))))

    for x in range(0,df_CIPE['Termo'].size):
        df_CIPE_stem.at[x,'Termo'] = removearticles1(unidecode.unidecode(stemmer(str(df_CIPE.at[x,'Termo']))))


    df_termos_match = pd.DataFrame(data=None, columns=['Termos', 'Regra', 'perc', 'ICNPcode', 'ICNPterm',
                                                       'ICNPmod', 'ICNPeixo', 'ICNPVersao'])    



    zz = 0
    achou = 0
    candidato = ''
    perc_max = 0
    metodo = 0
    y_aux = 0
    terms = 0

    controle = 1
    for x in range(0, len(df_termos_norm)):
        CUIS = []
        t = len(df_termos_norm.at[x,'Termos'])
        t_max = int(t+(t*0.2))
        t_min =  int(t-(t*0.2))
        t_lemma = len(df_termos_lemma.at[x, 'Termos'])
        t_max_lemma = int(t_lemma+(t_lemma*0.2))
        t_min_lemma =  int(t_lemma-(t_lemma*0.2))
        perc_max=0
        metodo = 0
        '''locTerm = CIPE_df.loc[CIPE_df['Termos'] == df_termos.at[x, 'Termos']]
        if not locTerm['Termos'].empty:
        #if df_termos.at[x, 'Termos'] == CIPE_df.at[zzz, 'Termos']:
            for xx in range(0,len(locTerm['Termos'])):
                df_termos_match.at[zz, 'Termos'] = CIPE_df.at[locTerm.index[xx], 'Termos']
                df_termos_match.at[zz, 'ICNPcode'] = CIPE_df.at[locTerm.index[xx], 'ICNPcode']
                df_termos_match.at[zz, 'ICNPterm'] = CIPE_df.at[locTerm.index[xx], 'ICNPterm']
                df_termos_match.at[zz, 'ICNPmod'] = CIPE_df.at[locTerm.index[xx], 'ICNPmod']
                df_termos_match.at[zz, 'ICNPVersao'] = CIPE_df.at[locTerm.index[xx], 'ICNPVersao']
                df_termos_match.at[zz, 'ICNPeixo'] = CIPE_df.at[locTerm.index[xx], 'ICNPeixo']
                df_termos_match.at[zz, 'Regra'] = CIPE_df.at[locTerm.index[xx], 'Regra']
                df_termos_match.at[zz, 'perc'] = CIPE_df.at[locTerm.index[xx], 'perc']
                zz += 1
            continue
        print(df_termos.at[x, 'Termos'], ' ------- passou')
        print(controle , ' termos ja passaram')'''
        controle += 1
        for y in range(0, len(df_CIPE_norm)):
            num = levenshtein(df_termos_norm.at[x, 'Termos'], df_CIPE_norm.at[y, 'Termo'])
            perc = 100 - num / len(df_CIPE_norm.at[y, 'Termo']) * 100
            if perc > perc_max:
                perc_max = perc
                candidato = df_CIPE_norm.at[y, 'Termo']
                metodo = 1
                y_aux = y
        if not perc_max == 100:
            #print(df_termos_norm.at[x,'Termos'],' entrou')
            noventa = False
            for y in range(0,len(df_umls_norm['STR'])):

                if df_termos_norm.at[x,'Termos'][0] == df_umls_norm.at[y,'STR'][0] and len(df_umls_norm.at[y,'STR']) < t_max and len(df_umls_norm.at[y,'STR']) > t_min:
                    num = levenshtein(df_termos_norm.at[x,'Termos'],df_umls_norm.at[y,'STR'])
                    if len(df_umls_norm.at[y,'STR']) > 0:
                        perc = 100-num/len(df_umls_norm.at[y,'STR'])*100
                    else:
                        perc = 0
                    if perc == 100:
                        if not df_umls_norm.at[y, 'CUI'] in CUIS:
                            CUIS.append(df_umls_norm.at[y, 'CUI'])
                            for z in range(0, len(df_umls_rel)):
                                if df_umls_rel.at[z,'CUI1'] == df_umls_norm.at[y, 'CUI']:
                                    if not df_umls_rel.at[z,'CUI2'] in CUIS:
                                        CUIS.append(df_umls_rel.at[z,'CUI2'])
                        print('achou')
                        noventa = False
                        terms += 1
                        break
                    if perc >= 90:
                        if not df_umls_norm.at[y, 'CUI'] in CUIS:
                            CUIS.append(df_umls_norm.at[y, 'CUI'])
                            for z in range(0, len(df_umls_rel)):
                                if df_umls_rel.at[z,'CUI1'] == df_umls_norm.at[y, 'CUI']:
                                    if not df_umls_rel.at[z,'CUI2'] in CUIS:
                                        CUIS.append(df_umls_rel.at[z,'CUI2'])
                        print('talvez achou')
                        noventa = True
            if noventa:
                terms+=1
            if perc < 90:
                noventa = False
            for y in range(0,len(df_umls_norm['STR'])):
                if df_termos_lemma.at[x, 'Termos'][0] == df_umls_norm.at[y, 'STRlemma'][0] and len(df_umls_norm.at[y, 'STRlemma']) < t_max_lemma and len(df_umls_norm.at[y, 'STRlemma']) > t_min_lemma:
                    num_lemma = levenshtein(df_termos_lemma.at[x, 'Termos'], df_umls_norm.at[y, 'STRlemma'])
                    num_steam = levenshtein(df_termos_stem.at[x, 'Termos'], df_umls_norm.at[y, 'STRstem'])
                    if len(df_umls_norm.at[y, 'STRlemma']) > 0:
                        perc_lemma = 100-num_lemma/len(df_umls_norm.at[y, 'STRlemma'])*100
                        perc_steam = 100-num_steam/len(df_umls_norm.at[y, 'STRstem'])*100
                    else:
                        perc_lemma = 0
                        perc_steam = 0
                    if perc_lemma == 100 and perc_steam >= 90:
                        if not df_umls_norm.at[y, 'CUI'] in CUIS:
                            CUIS.append(df_umls_norm.at[y, 'CUI'])
                            for z in range(0, len(df_umls_rel)):
                                if df_umls_rel.at[z,'CUI1'] == df_umls_norm.at[y, 'CUI']:
                                    if not df_umls_rel.at[z,'CUI2'] in CUIS:
                                        CUIS.append(df_umls_rel.at[z,'CUI2'])
                        print('achou')
                        noventa = False
                        terms += 1
                        break
                    if perc_lemma >= 90 and perc_steam >= 90:
                        if not df_umls_norm.at[y, 'CUI'] in CUIS:
                            CUIS.append(df_umls_norm.at[y, 'CUI'])
                            for z in range(0, len(df_umls_rel)):
                                if df_umls_rel.at[z,'CUI1'] == df_umls_norm.at[y, 'CUI']:
                                    if not df_umls_rel.at[z,'CUI2'] in CUIS:
                                        CUIS.append(df_umls_rel.at[z,'CUI2'])
                        print('talvez achou')
                        noventa = True
            if noventa:
                terms+=1
            if len(CUIS) > 0:
                print('existe na UMLS com as CUIs ', CUIS)
                for c in CUIS:
                    perc = 0
                    perc_max = 0
                    print(c)
                    locCUI = df_umls_norm.loc[df_umls_norm['CUI'] == c]
                    print(locCUI)
                    if not locCUI['CUI'].empty:
                        for xx in range(0,len(locCUI['CUI'])):
                            perc = 0
                            perc_max = 0
                            df_umls_norm.at[locCUI.index[xx], 'STR']
                            df_umls_norm.at[locCUI.index[xx], 'STRlemma']
                            df_umls_norm.at[locCUI.index[xx], 'STRstem']

                            for y in range(0, len(df_CIPE_norm)):
                                num = levenshtein(df_umls_norm.at[locCUI.index[xx], 'STR'], df_CIPE_norm.at[y, 'Termo'])
                                perc = 100 - num / len(df_CIPE_norm.at[y, 'Termo']) * 100
                                if perc > perc_max:
                                    perc_max = perc
                                    candidato = df_CIPE_norm.at[y, 'Termo']
                                    metodo = 1
                                    y_aux = y
                                if perc_max == 100:
                                    metodo = 1
                                    break
                            if metodo == 1 and perc_max >= 90:
                                print('encontrou o termo ', df_umls_norm.at[locCUI.index[xx], 'STR'], ' na CIPE o termo ', df_CIPE.at[y_aux, 'Termo'], ' com o codigo CIPE ',df_CIPE.at[y_aux, 'Código'])
                                df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                df_termos_match.at[zz, 'ICNPmod'] = df_umls_norm.at[locCUI.index[xx], 'STR']
                                df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                df_termos_match.at[zz, 'Regra'] = 1
                                df_termos_match.at[zz, 'perc'] = perc_max
                                zz += 1
                            if not perc_max == 100:
                                for y in range(0, len(df_CIPE_norm)):
                                    numlemma = levenshtein(df_umls_norm.at[locCUI.index[xx], 'STRlemma'], df_CIPE_lemma.at[y, 'Termo'])
                                    numstem = levenshtein(df_umls_norm.at[locCUI.index[xx], 'STRstem'], df_CIPE_stem.at[y, 'Termo'])
                                    perclemma = 100 - numlemma / len(df_CIPE_lemma.at[y, 'Termo']) * 100
                                    percstem = 100 - numstem / len(df_CIPE_lemma.at[y, 'Termo']) * 100
                                    if perclemma == 100 and percstem == 100:
                                        print('encontrou o termo ', df_umls_norm.at[locCUI.index[xx], 'STR'], ' na CIPE o termo ', df_CIPE.at[y_aux, 'Termo'], ' com o codigo CIPE ',df_CIPE.at[y_aux, 'Código'])
                                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y, 'Código']
                                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y, 'Termo']
                                        df_termos_match.at[zz, 'ICNPmod'] = df_umls_norm.at[locCUI.index[xx], 'STR']
                                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y, 'Versão']
                                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y, 'Eixo']
                                        df_termos_match.at[zz, 'Regra'] = 2
                                        df_termos_match.at[zz, 'perc'] = 100
                                        zz += 1
                                    elif perclemma >= 90 and percstem >= 90:
                                        print('encontrou o termo ', df_umls_norm.at[locCUI.index[xx], 'STR'], ' na CIPE o termo ', df_CIPE.at[y_aux, 'Termo'], ' com o codigo CIPE ',df_CIPE.at[y_aux, 'Código'])
                                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y, 'Código']
                                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y, 'Termo']
                                        df_termos_match.at[zz, 'ICNPmod'] = df_umls_norm.at[locCUI.index[xx], 'STR']
                                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y, 'Versão']
                                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y, 'Eixo']
                                        df_termos_match.at[zz, 'Regra'] = 2
                                        df_termos_match.at[zz, 'perc'] = (perclemma,percstem)
                                        zz += 1
    print('foram encontrados pelo metodo 1, ', terms,' termos')




    #writer = pd.ExcelWriter(primeiroArquivoExcel)
    #df_termos_match.to_excel(writer, 'Match')
    #writer.save()



    print(100 - 1 / len('lucass') * 100)

    zz = 0
    achou = 0
    candidato = ''
    perc_max = 0
    metodo = 0
    y_aux = 0

    controle = 1
    for x in range(0, len(df_termos_norm)):

        metodo = 0
        '''locTerm = CIPE_df.loc[CIPE_df['Termos'] == df_termos.at[x, 'Termos']]
        if not locTerm['Termos'].empty:
        #if df_termos.at[x, 'Termos'] == CIPE_df.at[zzz, 'Termos']:
            for xx in range(0,len(locTerm['Termos'])):
                df_termos_match.at[zz, 'Termos'] = CIPE_df.at[locTerm.index[xx], 'Termos']
                df_termos_match.at[zz, 'ICNPcode'] = CIPE_df.at[locTerm.index[xx], 'ICNPcode']
                df_termos_match.at[zz, 'ICNPterm'] = CIPE_df.at[locTerm.index[xx], 'ICNPterm']
                df_termos_match.at[zz, 'ICNPmod'] = CIPE_df.at[locTerm.index[xx], 'ICNPmod']
                df_termos_match.at[zz, 'ICNPVersao'] = CIPE_df.at[locTerm.index[xx], 'ICNPVersao']
                df_termos_match.at[zz, 'ICNPeixo'] = CIPE_df.at[locTerm.index[xx], 'ICNPeixo']
                df_termos_match.at[zz, 'Regra'] = CIPE_df.at[locTerm.index[xx], 'Regra']
                df_termos_match.at[zz, 'perc'] = CIPE_df.at[locTerm.index[xx], 'perc']
                zz += 1
            continue
        print(df_termos.at[x, 'Termos'], ' ------- passou')
        print(controle , ' termos ja passaram')'''
        controle += 1
        for y in range(0, len(df_CIPE_norm)):
            num = levenshtein(df_termos_norm.at[x, 'Termos'], df_CIPE_norm.at[y, 'Termo'])
            perc = 100 - num / len(df_CIPE_norm.at[y, 'Termo']) * 100
            if perc > perc_max:
                perc_max = perc
                candidato = df_CIPE_norm.at[y, 'Termo']
                metodo = 1
                y_aux = y
            if perc_max == 100:
                metodo = 1
                break
        if metodo == 1 and perc_max >= 90:
            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
            df_termos_match.at[zz, 'ICNPmod'] = candidato
            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
            df_termos_match.at[zz, 'Regra'] = 1
            df_termos_match.at[zz, 'perc'] = perc_max
            zz += 1
        if not perc_max == 100:
            for y in range(0, len(df_CIPE_norm)):
                num = levenshtein(df_termos_lemma.at[x, 'Termos'], df_CIPE_lemma.at[y, 'Termo'])
                perc = 100 - num / len(df_CIPE_lemma.at[y, 'Termo']) * 100
                if perc > perc_max:
                    perc_max = perc
                    candidato = df_CIPE_lemma.at[y, 'Termo']
                    metodo = 2
                    y_aux = y
                elif perc == 100 and perc_max == 100:
                    df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                    df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                    df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                    df_termos_match.at[zz, 'ICNPmod'] = candidato
                    df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                    df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                    df_termos_match.at[zz, 'Regra'] = 2
                    df_termos_match.at[zz, 'perc'] = perc_max
                    zz += 1
                    perc_max = perc
                    candidato = df_CIPE_lemma.at[y, 'Termo']
                    metodo = 2
                    y_aux = y
            if metodo == 2 and perc_max >= 90:
                df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                df_termos_match.at[zz, 'ICNPmod'] = candidato
                df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                df_termos_match.at[zz, 'Regra'] = 2
                df_termos_match.at[zz, 'perc'] = perc_max
                zz += 1
            #if not perc_max == 100:
            if not perc_max == 1000:
                perc_max = 0
                for y in range(0, len(df_CIPE_norm)):
                    num = levenshtein(df_termos_stem.at[x, 'Termos'], df_CIPE_stem.at[y, 'Termo'])
                    perc = 100 - num / len(df_CIPE_stem.at[y, 'Termo']) * 100
                    if perc > perc_max:
                        perc_max = perc
                        candidato = df_CIPE_stem.at[y, 'Termo']
                        metodo = 3
                        y_aux = y
                    elif perc == 100 and perc_max == 100:
                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                        df_termos_match.at[zz, 'Regra'] = 3
                        df_termos_match.at[zz, 'perc'] = perc_max
                        zz += 1
                        perc_max = perc
                        candidato = df_CIPE_lemma.at[y, 'Termo']
                        metodo = 3
                        y_aux = y
                if metodo == 3 and perc_max >= 90:
                    df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                    df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                    df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                    df_termos_match.at[zz, 'ICNPmod'] = candidato
                    df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                    df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                    df_termos_match.at[zz, 'Regra'] = 3
                    df_termos_match.at[zz, 'perc'] = perc_max
                    zz += 1
                #if perc_max < 90:
                if perc_max < 1000:
                    perc_max = 0
                    sem_sin = False
                    sinonimos_t = []
                    if len(df_termos.at[x, 'Termos'].split()) == 1:
                        try:
                            word = dicio.search(df_termos.at[x, 'Termos'])
                            for w in word.synonyms:
                                sinonimos_t.append(removearticles1(unidecode.unidecode(str(w))).strip())
                        except:
                            sem_sin = True
                    else:
                        iniR = time.time()
                        sin = {}
                        sin_lemma = {}
                        sin_steam = {}
                        words2 = (removearticles1(df_termos.at[x, 'Termos']).strip()).split()
                        print(words2)
                        for p in words2:
                            sinonimos = []
                            sinonimos_lemma = []
                            sinonimos_steam = []
                            try:
                                w = dicio.search(p)
                                sinonimos = w.synonyms
                                i = 0
                                for s in sinonimos:
                                    sinonimos[i] = unidecode.unidecode(str(s))
                                    i += 1
                                i = 0
                                for s in sinonimos:
                                    sinonimos_lemma.append(lematizador(str(s)))
                                    sinonimos_steam.append(stemmer(str(s)))
                                    i += 1
                                sinonimos.append(p)
                                sinonimos_lemma.append(lematizador(p))
                                sinonimos_steam.append(stemmer(p))
                                sin[p] = sinonimos
                                sin_lemma[p] = sinonimos_lemma
                                sin_steam[p] = sinonimos_steam
                            except:
                                sin[p]=[p]
                                sin_lemma[p] = [lematizador(p)]
                                sin_steam[p] = [stemmer(p)]
                        fimR = time.time()
                        print('tempo sinonimos, ', (fimR-iniR))
                        print('---------dicionarios-------', df_termos_norm.at[x,'Termos'])
                        print(sin)
                        print(sin_lemma)
                        print(sin_steam)


                        n = 0
                        first = True
                        for p in words2:
                            if first:
                                n = len(sin[p])
                                first = False
                                continue
                            else:
                                if len(sin[p]) != 0:
                                    n *= len(sin[p])
                                else:
                                    continue
                        print(df_termos_norm.at[x, 'Termos'], ' numero de combinações: ', n)
                        posib = []
                        posib_lemma = []
                        posib_steam = []


                        iniR = time.time()
                        first = True
                        for p in words2:
                            if first:
                                first = False
                                for s in sin[p]:
                                    posib.append(s)
                                for s in sin_lemma[p]:
                                    posib_lemma.append(s)
                                for s in sin_steam[p]:
                                    posib_steam.append(s)
                            else:
                                posib = comb(posib, sin[p])
                                posib_lemma = comb(posib_lemma, sin_lemma[p])
                                posib_steam = comb(posib_steam, sin_steam[p])
                        fimR = time.time()
                        print('tempo posib, ', (fimR-iniR))
                        print(posib)

                        if len(posib) > 1:
                            sinonimos_t = posib
                        else:
                            sem_sin = True

                    if not sem_sin:
                        for sin in sinonimos_t:
                            for y in range(0, len(df_CIPE_norm)):
                                num = levenshtein(str(sin), df_CIPE_norm.at[y, 'Termo'])
                                perc = 100 - num / len(df_CIPE_norm.at[y, 'Termo']) * 100
                                if perc > perc_max:
                                    perc_max = perc
                                    candidato = df_CIPE_norm.at[y, 'Termo']
                                    metodo = 12
                                    y_aux = y
                                if perc_max == 100:
                                    metodo = 12
                                    break
                            if not perc_max == 100:
                                for y in range(0, len(df_CIPE_norm)):
                                    num = levenshtein(lematizador(str(sin)), df_CIPE_lemma.at[y, 'Termo'])
                                    perc = 100 - num / len(df_CIPE_lemma.at[y, 'Termo']) * 100
                                    if perc > perc_max:
                                        perc_max = perc
                                        candidato = df_CIPE_lemma.at[y, 'Termo']
                                        metodo = 22
                                        y_aux = y
                                    elif perc == 100 and perc_max == 100:
                                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                        df_termos_match.at[zz, 'Regra'] = 22
                                        df_termos_match.at[zz, 'perc'] = perc_max
                                        zz += 1
                                        perc_max = perc
                                        candidato = df_CIPE_lemma.at[y, 'Termo']
                                        metodo = 22
                                        y_aux = y
                                if not perc_max == 100:
                                    for y in range(0, len(df_CIPE_norm)):
                                        num = levenshtein(stemmer(str(sin)), df_CIPE_stem.at[y, 'Termo'])
                                        perc = 100 - num / len(df_CIPE_stem.at[y, 'Termo']) * 100
                                        if perc > perc_max:
                                            perc_max = perc
                                            candidato = df_CIPE_stem.at[y, 'Termo']
                                            metodo = 32
                                            y_aux = y
                                        elif perc == 100 and perc_max == 100:
                                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                            df_termos_match.at[zz, 'ICNPmod'] = candidato
                                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                            df_termos_match.at[zz, 'Regra'] = 32
                                            df_termos_match.at[zz, 'perc'] = perc_max
                                            zz += 1
                                            perc_max = perc
                                            candidato = df_CIPE_lemma.at[y, 'Termo']
                                            metodo = 32
                                            y_aux = y
                        if metodo == 12 and perc_max >= 90:
                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                            df_termos_match.at[zz, 'ICNPmod'] = candidato
                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                            df_termos_match.at[zz, 'Regra'] = 12
                            df_termos_match.at[zz, 'perc'] = perc_max
                            zz += 1
                        elif metodo == 22 and perc_max >= 90:
                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                            df_termos_match.at[zz, 'ICNPmod'] = candidato
                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                            df_termos_match.at[zz, 'Regra'] = 22
                            df_termos_match.at[zz, 'perc'] = perc_max
                            zz += 1
                        elif metodo == 32 and perc_max >= 90:
                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                            df_termos_match.at[zz, 'ICNPmod'] = candidato
                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                            df_termos_match.at[zz, 'Regra'] = 32
                            df_termos_match.at[zz, 'perc'] = perc_max
                            zz += 1
                    #if not (metodo == 12 or metodo == 22 or metodo == 32):
                    if not (metodo == 122 or metodo == 222 or metodo == 322):
                        perc_max = 0
                        words = df_termos_norm.at[x, 'Termos'].split()
                        if len(words) > 1:
                            for y in range(0, len(df_CIPE_norm)):
                                word_comp = df_CIPE_norm.at[y, 'Termo'].split()
                                med_perc = 0
                                if len(words) < len(word_comp):
                                    subString = True
                                    for p in words:
                                        tem_word = False
                                        for p_comp in word_comp:
                                            num = levenshtein(p, p_comp)
                                            perc = 100 - num / len(p_comp) * 100
                                            if perc > 90:
                                                med_perc += perc
                                                tem_word = True
                                                break
                                        if not tem_word:
                                            subString = False
                                    if subString:
                                        perc_max = med_perc / len(words)
                                        candidato = df_CIPE_norm.at[y, 'Termo']
                                        metodo = 51
                                        y_aux = y
                                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                        df_termos_match.at[zz, 'Regra'] = 51
                                        df_termos_match.at[zz, 'perc'] = perc_max
                                        zz += 1
                                elif len(words) > len(word_comp):
                                    supString = True
                                    for p in word_comp:
                                        tem_word = False
                                        for p_comp in words:
                                            num = levenshtein(p, p_comp)
                                            perc = 100 - num / len(p_comp) * 100
                                            if perc > 90:
                                                med_perc += perc
                                                tem_word = True
                                                break
                                        if not tem_word:
                                            supString = False
                                    if supString:
                                        perc_max = med_perc / len(word_comp)
                                        candidato = df_CIPE_norm.at[y, 'Termo']
                                        metodo = 61
                                        y_aux = y
                                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                        df_termos_match.at[zz, 'Regra'] = 61
                                        df_termos_match.at[zz, 'perc'] = perc_max
                                        zz += 1
                        elif len(words) == 1:
                            for y in range(0, len(df_CIPE_norm)):
                                word_comp = df_CIPE_norm.at[y, 'Termo'].split()
                                med_perc = 0
                                if len(words) < len(word_comp):
                                    subString = True
                                    for p in words:
                                        tem_word = False
                                        for p_comp in word_comp:
                                            num = levenshtein(p, p_comp)
                                            perc = 100 - num / len(p_comp) * 100
                                            if perc > 90:
                                                med_perc += perc
                                                tem_word = True
                                                break
                                        if not tem_word:
                                            subString = False
                                    if subString:
                                        perc_max = med_perc / len(words)
                                        candidato = df_CIPE_norm.at[y, 'Termo']
                                        metodo = 51
                                        y_aux = y
                                        df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                        df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                        df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                        df_termos_match.at[zz, 'ICNPmod'] = candidato
                                        df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                        df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                        df_termos_match.at[zz, 'Regra'] = 51
                                        df_termos_match.at[zz, 'perc'] = perc_max
                                        zz += 1
                        if not (metodo == 51 or metodo == 61):
                            words = df_termos_lemma.at[x, 'Termos'].split()
                            if len(words) > 1:
                                for y in range(0, len(df_CIPE_norm)):
                                    word_comp = df_CIPE_lemma.at[y, 'Termo'].split()
                                    med_perc = 0
                                    if len(words) < len(word_comp):
                                        subString = True
                                        for p in words:
                                            tem_word = False
                                            for p_comp in word_comp:
                                                num = levenshtein(p, p_comp)
                                                perc = 100 - num / len(p_comp) * 100
                                                if perc > 90:
                                                    med_perc += perc
                                                    tem_word = True
                                                    break
                                            if not tem_word:
                                                subString = False
                                        if subString:
                                            perc_max = med_perc / len(words)
                                            candidato = df_CIPE_norm.at[y, 'Termo']
                                            metodo = 52
                                            y_aux = y
                                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                            df_termos_match.at[zz, 'ICNPmod'] = candidato
                                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                            df_termos_match.at[zz, 'Regra'] = 52
                                            df_termos_match.at[zz, 'perc'] = perc_max
                                            zz += 1
                                    elif len(words) > len(word_comp):
                                        supString = True
                                        for p in word_comp:
                                            tem_word = False
                                            for p_comp in words:
                                                num = levenshtein(p, p_comp)
                                                perc = 100 - num / len(p_comp) * 100
                                                if perc > 90:
                                                    med_perc += perc
                                                    tem_word = True
                                                    break
                                            if not tem_word:
                                                supString = False
                                        if supString:
                                            perc_max = med_perc / len(word_comp)
                                            candidato = df_CIPE_norm.at[y, 'Termo']
                                            metodo = 62
                                            y_aux = y
                                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                            df_termos_match.at[zz, 'ICNPmod'] = candidato
                                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                            df_termos_match.at[zz, 'Regra'] = 62
                                            df_termos_match.at[zz, 'perc'] = perc_max
                                            zz += 1
                            elif len(words) == 1:
                                for y in range(0, len(df_CIPE_norm)):
                                    word_comp = df_CIPE_lemma.at[y, 'Termo'].split()
                                    med_perc = 0
                                    if len(words) < len(word_comp):
                                        subString = True
                                        for p in words:
                                            tem_word = False
                                            for p_comp in word_comp:
                                                num = levenshtein(p, p_comp)
                                                perc = 100 - num / len(p_comp) * 100
                                                if perc > 90:
                                                    med_perc += perc
                                                    tem_word = True
                                                    break
                                            if not tem_word:
                                                subString = False
                                        if subString:
                                            perc_max = med_perc / len(words)
                                            candidato = df_CIPE_norm.at[y, 'Termo']
                                            metodo = 52
                                            y_aux = y
                                            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                            df_termos_match.at[zz, 'ICNPmod'] = candidato
                                            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                            df_termos_match.at[zz, 'Regra'] = 52
                                            df_termos_match.at[zz, 'perc'] = perc_max
                                            zz += 1
                            if not (metodo == 52 or metodo == 62):
                                words = df_termos_stem.at[x, 'Termos'].split()
                                if len(words) > 1:
                                    for y in range(0, len(df_CIPE_norm)):
                                        word_comp = df_CIPE_stem.at[y, 'Termo'].split()
                                        med_perc = 0
                                        if len(words) < len(word_comp):
                                            subString = True
                                            for p in words:
                                                tem_word = False
                                                for p_comp in word_comp:
                                                    num = levenshtein(p, p_comp)
                                                    perc = 100 - num / len(p_comp) * 100
                                                    if perc > 90:
                                                        med_perc += perc
                                                        tem_word = True
                                                        break
                                                if not tem_word:
                                                    subString = False
                                            if subString:
                                                perc_max = med_perc / len(words)
                                                candidato = df_CIPE_norm.at[y, 'Termo']
                                                metodo = 53
                                                y_aux = y
                                                df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                                df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                                df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                                df_termos_match.at[zz, 'ICNPmod'] = candidato
                                                df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                                df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                                df_termos_match.at[zz, 'Regra'] = 53
                                                df_termos_match.at[zz, 'perc'] = perc_max
                                                zz += 1
                                        elif len(words) > len(word_comp):
                                            supString = True
                                            for p in word_comp:
                                                tem_word = False
                                                for p_comp in words:
                                                    num = levenshtein(p, p_comp)
                                                    perc = 100 - num / len(p_comp) * 100
                                                    if perc > 90:
                                                        med_perc += perc
                                                        tem_word = True
                                                        break
                                                if not tem_word:
                                                    supString = False
                                            if supString:
                                                perc_max = med_perc / len(word_comp)
                                                candidato = df_CIPE_norm.at[y, 'Termo']
                                                metodo = 63
                                                y_aux = y
                                                df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                                df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                                df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                                df_termos_match.at[zz, 'ICNPmod'] = candidato
                                                df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                                df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                                df_termos_match.at[zz, 'Regra'] = 63
                                                df_termos_match.at[zz, 'perc'] = perc_max
                                                zz += 1
                                elif len(words) == 1:
                                    for y in range(0, len(df_CIPE_norm)):
                                        word_comp = df_CIPE_stem.at[y, 'Termo'].split()
                                        med_perc = 0
                                        if len(words) < len(word_comp):
                                            subString = True
                                            for p in words:
                                                tem_word = False
                                                for p_comp in word_comp:
                                                    num = levenshtein(p, p_comp)
                                                    perc = 100 - num / len(p_comp) * 100
                                                    if perc > 90:
                                                        med_perc += perc
                                                        tem_word = True
                                                        break
                                                if not tem_word:
                                                    subString = False
                                            if subString:
                                                perc_max = med_perc / len(words)
                                                candidato = df_CIPE_norm.at[y, 'Termo']
                                                metodo = 53
                                                y_aux = y
                                                df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
                                                df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
                                                df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
                                                df_termos_match.at[zz, 'ICNPmod'] = candidato
                                                df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
                                                df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
                                                df_termos_match.at[zz, 'Regra'] = 53
                                                df_termos_match.at[zz, 'perc'] = perc_max
                                                zz += 1
        '''if metodo == 1 and perc_max >= 90:
            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
            df_termos_match.at[zz, 'ICNPmod'] = candidato
            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
            df_termos_match.at[zz, 'Regra'] = 1
            df_termos_match.at[zz, 'perc'] = perc_max
            zz += 1
        elif metodo == 2 and perc_max >= 90:
            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
            df_termos_match.at[zz, 'ICNPmod'] = candidato
            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
            df_termos_match.at[zz, 'Regra'] = 2
            df_termos_match.at[zz, 'perc'] = perc_max
            zz += 1
        elif metodo == 3 and perc_max >= 90:
            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
            df_termos_match.at[zz, 'ICNPcode'] = df_CIPE.at[y_aux, 'Código']
            df_termos_match.at[zz, 'ICNPterm'] = df_CIPE.at[y_aux, 'Termo']
            df_termos_match.at[zz, 'ICNPmod'] = candidato
            df_termos_match.at[zz, 'ICNPVersao'] = df_CIPE.at[y_aux, 'Versão']
            df_termos_match.at[zz, 'ICNPeixo'] = df_CIPE.at[y_aux, 'Eixo']
            df_termos_match.at[zz, 'Regra'] = 3
            df_termos_match.at[zz, 'perc'] = perc_max
            zz += 1
        elif not (metodo == 51 or metodo == 61 or metodo == 52 or metodo == 62 or metodo == 53 or metodo == 63
                  or metodo == 12 or metodo == 22 or metodo == 32):
            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
            df_termos_match.at[zz, 'Regra'] = 0
            df_termos_match.at[zz, 'perc'] = 0
            df_termos_match.at[zz, 'ICNPcode'] = ''
            df_termos_match.at[zz, 'ICNPterm'] = ''
            df_termos_match.at[zz, 'ICNPVersao'] = ''
            df_termos_match.at[zz, 'ICNPeixo'] = ''
            zz += 1
        else:
            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
            df_termos_match.at[zz, 'Regra'] = 0
            df_termos_match.at[zz, 'perc'] = 0
            df_termos_match.at[zz, 'ICNPcode'] = ''
            df_termos_match.at[zz, 'ICNPterm'] = ''
            df_termos_match.at[zz, 'ICNPVersao'] = ''
            df_termos_match.at[zz, 'ICNPeixo'] = ''
            zz += 1'''
        if perc_max < 90:
            df_termos_match.at[zz, 'Termos'] = df_termos.at[x, 'Termos']
            df_termos_match.at[zz, 'Regra'] = 0
            df_termos_match.at[zz, 'perc'] = 0
            df_termos_match.at[zz, 'ICNPcode'] = ''
            df_termos_match.at[zz, 'ICNPterm'] = ''
            df_termos_match.at[zz, 'ICNPVersao'] = ''
            df_termos_match.at[zz, 'ICNPeixo'] = ''
            zz += 1
        print(df_termos_norm.at[x, 'Termos'], ' tem o par ', candidato, ' com a porcentagem ', perc_max)
        perc_max = 0




    writer = pd.ExcelWriter(OutputFile)
    df_termos_match.to_excel(writer, 'Match')
    writer.save()

    print("Numero de termos:",df_termos_norm['Termos'].size)
    print("Achou:   ",achou)
    print("Achou(%):",(achou/df_termos_norm['Termos'].size)*100)


In [ ]:
%%time
#ver se é melhorar separar os arquivos, ou só precisa de um:
TermsFile = 'TermosMapClin.xlsx'
CIPEFile   = 'CIPE2017_tabela organizada_CPeCC.xlsx'

MRCONSOFILE = "MRCONSO.RRF"
MRXW_PORFILE = "MRXW_POR.RRF"
MRRELFILE = "MRREL.RRF"

OutputFile = 'CliniMap_CIPE_Resultado.xlsx'

MapClinCIPE(TermsFile,CIPEFile,MRCONSOFILE,MRXW_PORFILE,MRRELFILE,OutputFile)